In [9]:
# importamos las librerias necesarias
import pandas as pd
import numpy as np

# importamos los datos
amazon = pd.read_csv('./data/amazon_prime_titles.csv')
disney = pd.read_csv('./data/disney_plus_titles.csv')
hulu = pd.read_csv('./data/hulu_titles.csv')
netflix = pd.read_csv('./data/netflix_titles.csv')

# eliminamos las columnas que no necesitamos para que los dataframes sean similares
netflix.drop(columns=netflix.columns[12:], inplace=True)
names = ['amazon', 'disney', 'hulu', 'netflix']
dataframes = [amazon, disney, hulu, netflix]

In [10]:
# Eliminamos los valores nulos de las columnas que nos interesan
for dataframe in dataframes:
    dataframe.dropna(subset=['title', 'listed_in', 'description'], inplace=True)

# Creamos una columna con los tags (palabras clave) de cada pelicula
for dataframe in dataframes:
    dataframe['tags'] = dataframe['description'] + ' ' + dataframe['listed_in']

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000, stop_words='english')

# Creamos una lista de vectores con las palabras clave de cada pelicula
vectores = []
for dataframe in dataframes:
    vectores.append(cv.fit_transform(dataframe['tags'].values.astype('U')).toarray())


In [12]:
# Hallamos la similitud entre los vectores de cada pelicula
from sklearn.metrics.pairwise import cosine_similarity
similarities = [cosine_similarity(vectores[i]) for i in range(4)]

In [13]:
# Guardamos los resultados en archivos pickle para no tener que volver a correr el codigo
# Antes se debe crear el directorio de saved 

import pickle
for i in range(4):
    with open(f'./saved/sim_{names[i]}.pkl', 'wb') as f:
        pickle.dump(similarities[i], f)
    
    df = dataframes[i][['title', 'tags']]
    with open(f'./saved/df_{names[i]}.pkl', 'wb') as f:
        pickle.dump(df, f)

In [14]:
df = pickle.load(open('./saved/df_netflix.pkl', 'rb'))
sim = pickle.load(open('./saved/sim_netflix.pkl', 'rb'))

def recommend(movies):
    index = df[df['title'] == movies].index[0]
    distance = sorted(list(enumerate(sim[index])), reverse=True, key=lambda x: x[1])
    for i in distance[1:6]:
        print(df.iloc[i[0]].title)


recommend('Pokémon the Movie: I Choose You!')

Pokémon the Movie: Power of Us
Pokémon: Mewtwo Strikes Back - Evolution
Pick of the Litter
Beethoven's 2nd
Motu Patlu: Deep Sea Adventure
